# Dynamo DB Docstore 演示

本指南向您展示如何直接使用由DynamoDB支持的`DocumentStore`抽象。通过将节点放入文档存储库，这使您能够在相同的基础文档存储库上定义多个索引，而不是在索引之间复制数据。

<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/docstore/DynamoDBDocstoreDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


如果您在colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
%pip install llama-index-storage-docstore-dynamodb
%pip install llama-index-storage-index-store-dynamodb
%pip install llama-index-vector-stores-dynamodb
%pip install llama-index-llms-openai

In [ ]:
!pip install llama-index

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex, SimpleKeywordTableIndex
from llama_index.core import SummaryIndex
from llama_index.llms.openai import OpenAI
from llama_index.core.response.notebook_utils import display_response
from llama_index.core import Settings

#### 下载数据


In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

### 加载文档


In [ ]:
reader = SimpleDirectoryReader("./data/paul_graham/")
documents = reader.load_data()

### 解析到节点


In [ ]:
from llama_index.core.node_parser import SentenceSplitter

nodes = SentenceSplitter().get_nodes_from_documents(documents)

### 加入Docstore


In [ ]:
TABLE_NAME = os.environ["DYNAMODB_TABLE_NAME"]

In [ ]:
from llama_index.storage.docstore.dynamodb import DynamoDBDocumentStore
from llama_index.storage.index_store.dynamodb import DynamoDBIndexStore
from llama_index.vector_stores.dynamodb import DynamoDBVectorStore

In [ ]:
storage_context = StorageContext.from_defaults(
    docstore=DynamoDBDocumentStore.from_table_name(table_name=TABLE_NAME),
    index_store=DynamoDBIndexStore.from_table_name(table_name=TABLE_NAME),
    vector_store=DynamoDBVectorStore.from_table_name(table_name=TABLE_NAME),
)

In [ ]:
storage_context.docstore.add_documents(nodes)

#### 定义和添加多个索引

每个索引使用相同的基础节点。


In [ ]:
# https://gpt-index.readthedocs.io/en/latest/api_reference/indices/list.html
# 创建一个SummaryIndex对象，传入节点和存储上下文作为参数
summary_index = SummaryIndex(nodes, storage_context=storage_context)

In [ ]:
# https://gpt-index.readthedocs.io/en/latest/api_reference/indices/vector_store.html
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)

In [ ]:
# https://gpt-index.readthedocs.io/en/latest/api_reference/indices/table.html
# 创建一个简单的关键词表索引
keyword_table_index = SimpleKeywordTableIndex(
    nodes, storage_context=storage_context
)

In [ ]:
# 注意：docstore仍然具有相同的节点
len(storage_context.docstore.docs)

#### 测试保存和加载


In [ ]:
# 注意：默认情况下，docstore、index_store和vector_index在创建时会持久化到DynamoDB中
# 注意：您也可以使用以下命令将简单的向量存储持久化到磁盘
storage_context.persist()

In [ ]:
# 记录索引ID
list_id = summary_index.index_id  # 摘要索引ID
vector_id = vector_index.index_id  # 向量索引ID
keyword_id = keyword_table_index.index_id  # 关键词表索引ID

In [ ]:
from llama_index.core import load_index_from_storage

# 重新创建存储上下文
storage_context = StorageContext.from_defaults(
    docstore=DynamoDBDocumentStore.from_table_name(table_name=TABLE_NAME),
    index_store=DynamoDBIndexStore.from_table_name(table_name=TABLE_NAME),
    vector_store=DynamoDBVectorStore.from_table_name(table_name=TABLE_NAME),
)

summary_index = load_index_from_storage(
    storage_context=storage_context, index_id=list_id
)
keyword_table_index = load_index_from_storage(
    storage_context=storage_context, index_id=keyword_id
)

# 需要在StorageContext中添加"vector_store=DynamoDBVectorStore.from_table_name(table_name=TABLE_NAME)"，以从DynamoDB加载向量索引
vector_index = load_index_from_storage(
    storage_context=storage_context, index_id=vector_id
)

#### 测试一些查询语句


In [ ]:
chatgpt = OpenAI(temperature=0, model="gpt-3.5-turbo")

Settings.llm = chatgpt
Settings.chunk_size = 1024

In [ ]:
query_engine = summary_index.as_query_engine()
list_response = query_engine.query("What is a summary of this document?")

In [ ]:
display_response(list_response)

In [ ]:
query_engine = vector_index.as_query_engine()
vector_response = query_engine.query("What did the author do growing up?")

In [ ]:
display_response(vector_response)

In [ ]:
query_engine = keyword_table_index.as_query_engine()
keyword_response = query_engine.query(
    "What did the author do after his time at YC?"
)

In [ ]:
display_response(keyword_response)